In [41]:
import pandas as pd
import numpy as np
# from sklearn.linear_model import LogisticRegression

from tensorflow.keras.preprocessing.text import Tokenizer


In [42]:
df_train = pd.read_csv("Y:/GP/Datasets/Emotion datasets/archive/train.txt",delimiter=';',names=['text','label'])
df_val = pd.read_csv("Y:/GP/Datasets/Emotion datasets/archive/val.txt",delimiter=';',names=['text','label'])

In [43]:
df = pd.concat([df_train,df_val])
df.reset_index(inplace=True,drop=True)
print("Shape of the DataFrame:",df.shape)
df.sample(5)

Shape of the DataFrame: (18000, 2)


,text,label
17477,i had finished my first leg the toughest longe...,love
2982,i feel truly blessed to have had the opportuni...,love
15484,i should ask them to move but the movers were ...,anger
7573,i know it signifies him feeling not dangerous ...,anger
6641,i feel like there is so much more i could be d...,love


In [44]:
def custom_encoder(df):
    df.replace(to_replace ="surprise", value =1, inplace=True)
    df.replace(to_replace ="love", value =1, inplace=True)
    df.replace(to_replace ="joy", value =1, inplace=True)
    df.replace(to_replace ="fear", value =0, inplace=True)
    df.replace(to_replace ="anger", value =0, inplace=True)
    df.replace(to_replace ="sadness", value =0, inplace=True)

custom_encoder(df['label'])
    

In [45]:
df.sample()

,text,label
16871,i feel sure is greater to those who are not da...,1


In [46]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [76]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index) + 1


In [81]:
vocab_size

16197

In [77]:
# train_vectors.shape
encoded_docs = tokenizer.texts_to_sequences(df['text'])

padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding


embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

# print(model.summary())

In [79]:
history = model.fit(padded_sequence,df['label'],validation_split=0.2, epochs=15, batch_size=64)


Epoch 1/15
225/225 [==============================] - 34s 142ms/step - loss: 0.6163 - accuracy: 0.6421 - val_loss: 0.4226 - val_accuracy: 0.8344
Epoch 2/15
225/225 [==============================] - 31s 138ms/step - loss: 0.2519 - accuracy: 0.9076 - val_loss: 0.1796 - val_accuracy: 0.9378
Epoch 3/15
225/225 [==============================] - 32s 140ms/step - loss: 0.1416 - accuracy: 0.9494 - val_loss: 0.1572 - val_accuracy: 0.9461
Epoch 4/15
225/225 [==============================] - 32s 140ms/step - loss: 0.0874 - accuracy: 0.9685 - val_loss: 0.1625 - val_accuracy: 0.9439
Epoch 5/15
225/225 [==============================] - 32s 141ms/step - loss: 0.0708 - accuracy: 0.9758 - val_loss: 0.1577 - val_accuracy: 0.9478
Epoch 6/15
225/225 [==============================] - 34s 152ms/step - loss: 0.0580 - accuracy: 0.9803 - val_loss: 0.1723 - val_accuracy: 0.9469
Epoch 7/15
225/225 [==============================] - 32s 141ms/step - loss: 0.0503 - accuracy: 0.9830 - val_loss: 0.1707 - val_ac

In [87]:
from keras.models import load_model
    
model.save('sentimentAnalysisSmallLSTM.h5')

In [98]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    print(tw)
    tw = pad_sequences(tw,maxlen=200)
    print(tw)
    prediction = int(model.predict(tw).round().item())
    print("Predicted label: ", prediction)



In [103]:
# predict_sentiment('I cried alot')

[[1, 1746, 1887]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0

In [88]:
# tokenizer2=Tokenizer()
# tokenizer2.fit_on_texts(df['text'])
# loaded_model = load_model('sentimentAnalysisSmallLSTM.h5')


In [92]:
# tw = tokenizer2.texts_to_sequences(['i feel like crying'])
# print(tw)
# tw = pad_sequences(tw,maxlen=200)
# print(tw)
# prediction = int(loaded_model.predict(tw).round().item())
# print("Predicted label: ", prediction)


[[1, 2, 13, 975]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   2
   13 975]]
1/1 [==============================] - 0s 39ms/step
Predicted label:  1


In [56]:
# from sklearn.feature_extraction.text import TfidfVectorizer# Create feature vectors
# vectorizer = TfidfVectorizer(min_df = 5,
#                              max_df = 0.8,
#                              sublinear_tf = True,
#                              use_idf = True)
# train_vectors = vectorizer.fit_transform(df['text'])

In [57]:
# test_df =  pd.read_csv("Y:/GP/Datasets/Emotion datasets/archive/test.txt",delimiter=';',names=['text','label'])

In [58]:
# X_test,y_test = test_df.text,test_df.label
# # encode the labels into two classes , 0 and 1
# custom_encoder(y_test)


# test_vectors = vectorizer.transform(X_test)



In [59]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report,plot_confusion_matrix,ConfusionMatrixDisplay

In [60]:
# from sklearn import svm
# # from sklearn.metrics import classification_report# Perform classification with SVM, kernel=linear
# classifier_linear = svm.SVC(kernel='linear')
# classifier_linear.fit(train_vectors, df['label'])
# prediction_linear = classifier_linear.predict(test_vectors)


In [61]:
# test_df.head()

In [62]:
report = classification_report(y_test, prediction_linear, output_dict=True)
print('positive: ', report['1'])
print('negative: ', report['0'])

NameError: name 'y_test' is not defined

In [ ]:
# plot_confusion_matrix(y_test,prediction_linear)

In [ ]:
# prediction_linear=logisticPrediction
acc_score = accuracy_score(y_test,prediction_linear)
pre_score = precision_score(y_test,prediction_linear)
rec_score = recall_score(y_test,prediction_linear)
print('Accuracy_score: ',acc_score)
print('Precision_score: ',pre_score)
print('Recall_score: ',rec_score)
print("-"*50)
cr = classification_report(y_test,prediction_linear)
print(cr)

Accuracy_score:  0.9735
Precision_score:  0.9737704918032787
Recall_score:  0.9684782608695652
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1080
           1       0.97      0.97      0.97       920

    accuracy                           0.97      2000
   macro avg       0.97      0.97      0.97      2000
weighted avg       0.97      0.97      0.97      2000



SVM linear : 97.35 .

svm rbf : 96.85.

svm sigmoid : 97.05.

Logistic Regression: 95.7.
